# Detecting Human-AI Generated Texts: A Multi-Method Classification Approach with Compact Style Embeddings

In [93]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Subset
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import Dataset

import pytorch_lightning as pl
from pytorch_lightning import Trainer, loggers
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from torchmetrics import Accuracy, F1Score

from transformers import DistilBertModel, DistilBertTokenizer, MobileBertModel, MobileBertTokenizer, PreTrainedTokenizer,MobileBertForSequenceClassification

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import numpy as np
from datasets import load_dataset

from tokenizers import Tokenizer
import random

from cpus import numcores
import pandas as pd
import os
import json

In [2]:
workers = numcores()
print(f'This device has {workers} cores.')

This device has 10 cores.


In [3]:
# set the device
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
elif torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
else:
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
    DEVICE = torch.device("cpu")
    print("Warning: You are using CPU. For better performance, use GPU.")
print("Pytorch version is: ", torch.__version__)
print("You are using: ", DEVICE)

Pytorch version is:  2.2.2+cu121
You are using:  cuda


## Tokenization and Model Initialization
We’ll define a function to initialize the tokenizer and model based on the choice.

In [4]:
def initialize_model(model_name='distilbert'):
    if model_name == 'mobilebert':
        tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
        model = MobileBertModel.from_pretrained('google/mobilebert-uncased').to(DEVICE)
    else:  # Default to DistilBERT
        tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(DEVICE)
    return tokenizer, model

#tokenizer, bert_model = initialize_model('distilbert')  # or 'mobilebert'
tokenizer, bert_model = initialize_model('mobilebert') 

## Load Dataset

### Mixset

The training set consists of 3,000 entries and the test set has 600 entries. 83.3\% of this dataset is MGT, and the rest of it is HWT.

In [5]:
# mixset = load_dataset("shuaishuaicdp/MixSet")

In [6]:
def read_json_file(file_path):
    # Open and load the JSON file
    with open(file_path, 'r', encoding='utf-8') as file:
        file_data = json.load(file)
    
    # Extract the 'revised' and 'binary' fields and store them in a list
    data = [(item['revised'], item['binary']) for item in file_data]

    # Convert list to DataFrame
    return pd.DataFrame(data, columns=['Revised', 'Binary'])

In [7]:
# Usage
data_folder = os.path.join(os.getcwd(), 'data')  # Construct the path to the data folder
test_file_path = os.path.join(data_folder, 'Mixset_test.json')  # Path to the test JSON file
train_file_path = os.path.join(data_folder, 'Mixset_train.json')  # Path to the train JSON file

# Read files and create DataFrames
mixset_test = read_json_file(test_file_path)
mixset_train = read_json_file(train_file_path)

Data Example:
```
{
        "category": "speech",
        "id": 3000,
        "original": "Two twin domes, two radically opposed design cultures. One is made of thousands of steel parts, the other of a single silk thread. One is synthetic, the other organic. One is imposed on the environment, the other creates it. One is designed for nature, the other is designed by her.michelangelo said that when he looked at raw marble, he saw a figure struggling to be free. The chisel was michelangelo's only tool. But living things are not chiseled. They grow. And in our smallest units of life, our cells, we carry all the information that's required for every other cell to function and to replicate.tools also have consequences. At least since the industrial revolution, the world of design has been dominated by the rigors of manufacturing and mass production.",
        "revised": "Two identical domes, two fundamentally different design philosophies. One consists of thousands of metallic components, while the other is crafted from a singular silken strand. One is artificial, the other natural. One imposes itself upon the environment, while the other is created by it. One is designed for nature, the other is designed by Nature herself. Michelangelo believed that when he gazed upon unhewn marble, he witnessed a form yearning to be liberated. His chisel served as the instrument to release its inherent beauty. However, living entities do not conform to such sculptural constraints; they develop and flourish through growth. Within our minuscule biological building blocks, known as cells, resides all the necessary knowledge for their own operation and propagation, as well as that of every other cell. Tools possess implications beyond their intended use. Since the advent of the Industrial Revolution, the realm of design has been governed by the principles of fabrication and large-scale production.",
        "mixset_category": "2llama_polish_token",
        "binary": "MGT"
    }
```


However, the scale of the MixSet Dataset is relatively small. Thus we downloaded `Augmented data for LLM - Detect AI Generated Text` dataset from kaggle, which consists of pure AI and pure human text. We will draw samples from llm dataset to make mixset a balance dataset, with 3 classes: pure human, pure AI, and Mix.

In [8]:
# LLM dataset


train_csv = pd.read_csv(os.path.join(data_folder, 'final_train.csv'))
test_csv = pd.read_csv(os.path.join(data_folder, 'final_test.csv'))

In [9]:
# Count of mixed texts (MGT) and human texts (HWT) in mixset
mix_train_counts = mixset_train['Binary'].value_counts()
mix_test_counts = mixset_test['Binary'].value_counts()

# Assume the number of mixed texts to be fixed and define the sample sizes
num_mixed = mix_train_counts['MGT']
num_human_hwt = mix_train_counts['HWT']

# Sample pure AI and pure human texts from CSV dataset
pure_ai_train = train_csv[train_csv['label'] == 1].sample(n=num_mixed, random_state=42)
pure_human_train = train_csv[train_csv['label'] == 0].sample(n=(num_mixed - num_human_hwt), random_state=42)

pure_ai_test = test_csv[test_csv['label'] == 1].sample(n=mix_test_counts['MGT'], random_state=42)
pure_human_test = test_csv[test_csv['label'] == 0].sample(n=(mix_test_counts['MGT'] - mix_test_counts['HWT']), random_state=42)

# Combine the datasets
combined_train = pd.concat([
    pd.DataFrame({'Text': pure_ai_train['text'], 'Label': 'PureAI'}),
    pd.DataFrame({'Text': pure_human_train['text'], 'Label': 'PureHuman'}),
    pd.DataFrame({'Text': mixset_train[mixset_train['Binary'] == 'MGT']['Revised'], 'Label': 'Mixed'}),
    pd.DataFrame({'Text': mixset_train[mixset_train['Binary'] == 'HWT']['Revised'], 'Label': 'PureHuman'})
])

combined_test = pd.concat([
    pd.DataFrame({'Text': pure_ai_test['text'], 'Label': 'PureAI'}),
    pd.DataFrame({'Text': pure_human_test['text'], 'Label': 'PureHuman'}),
    pd.DataFrame({'Text': mixset_test[mixset_test['Binary'] == 'MGT']['Revised'], 'Label': 'Mixed'}),
    pd.DataFrame({'Text': mixset_test[mixset_test['Binary'] == 'HWT']['Revised'], 'Label': 'PureHuman'})
])

# Shuffle the datasets to mix the entries properly
combined_train = combined_train.sample(frac=1, random_state=42).reset_index(drop=True)
combined_test = combined_test.sample(frac=1, random_state=42).reset_index(drop=True)

We randomly draw 5\% of the training set as validation set.

In [10]:
def create_train_validation_split(dataset, val_size=0.05, random_seed=42):
    """
    Splits the DataFrame into training and validation DataFrames in a stratified manner,
    ensuring that each class is represented in the validation set in proportion to its occurrence in the full dataset.

    Args:
        dataset (pandas.DataFrame): The complete dataset from which to create subsets.
        val_size (float, optional): The proportion of the dataset to use for validation.
        random_seed (int, optional): A seed for the random number generator for reproducibility.

    Returns:
        Tuple[pandas.DataFrame, pandas.DataFrame]: Training and validation DataFrames.
    """
    # Check if the dataset has a column named 'Label'
    if 'Label' not in dataset.columns:
        raise ValueError("Dataset must have a 'Label' column for stratified splitting")

    # Calculate the number of samples to include in each set using the specified validation size
    train_df, val_df = train_test_split(
        dataset, 
        test_size=val_size, 
        random_state=random_seed, 
        stratify=dataset['Label']
    )

    return train_df, val_df

In [11]:
combined_train, combined_val = create_train_validation_split(combined_train)

## Creating the Custom Dataset Class
We will creating a custom dataset class for the triplet model. It requires organizing our data into triplets in a way that each triplet consists of texts that are likely to have similar style characteristics but different content. Since the goal is to identify whether a text is from a human, AI, or a mixture, we need to structure the triplet as follows:

- Anchor (A1): A sample from any class.
- Positive (A2): Another sample from the same class but different content.
- Negative (B): A sample from a different class.

Since we are using pretrained models (e.g., DistilBERT, MobileBERT) and focusing on style, we decided keeping the stopwords to preserve the full stylistic and semantic integrity of the texts.

In [12]:
class TripletTextDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512, seed=None):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = data['Text']
        self.labels = data['Label']
        self.seed = seed
        
        # Use a dictionary to map text labels to integers
        self.label_mapping = {'PureAI': 0, 'PureHuman': 1, 'Mixed': 2}
        
        # Group indices for each class
        self.class_indices = {'PureAI': [], 'PureHuman': [], 'Mixed': []}
        for index, label in enumerate(self.labels):
            self.class_indices[label].append(index)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        if self.seed is not None:
            random.seed(self.seed)

        anchor_text = self.texts.iloc[idx]
        anchor_label = self.labels.iloc[idx]

        # Map the label string to an integer
        mapped_label = self.label_mapping[anchor_label]

        pos_index = idx
        while pos_index == idx:  # Ensure a different text for positive example
            pos_index = random.choice(self.class_indices[anchor_label])
        positive_text = self.texts.iloc[pos_index]

        neg_label = random.choice([l for l in self.label_mapping if l != anchor_label])
        neg_index = random.choice(self.class_indices[neg_label])
        negative_text = self.texts.iloc[neg_index]

        # Tokenize texts
        anchor_enc = self.tokenizer(anchor_text, add_special_tokens=True, 
                                    max_length=self.max_length, truncation=True,
                                    padding='max_length', return_tensors='pt')
        positive_enc = self.tokenizer(positive_text, add_special_tokens=True, 
                                      max_length=self.max_length, truncation=True,
                                      padding='max_length', return_tensors='pt')
        negative_enc = self.tokenizer(negative_text, add_special_tokens=True, 
                                      max_length=self.max_length, truncation=True,
                                      padding='max_length', return_tensors='pt')

        return {
            'anchor': anchor_enc['input_ids'].squeeze(0),
            'positive': positive_enc['input_ids'].squeeze(0),
            'negative': negative_enc['input_ids'].squeeze(0),
            'anchor_mask': anchor_enc['attention_mask'].squeeze(0),
            'positive_mask': positive_enc['attention_mask'].squeeze(0),
            'negative_mask': negative_enc['attention_mask'].squeeze(0),
            'labels': torch.tensor(mapped_label)  # Now this is a valid tensor conversion
        }


we will leverage PyTorch's DataLoader to greatly simplify sampling and batching.

In [13]:
train_dataset = TripletTextDataset(combined_train, tokenizer)
val_dataset = TripletTextDataset(combined_val, tokenizer)
test_dataset = TripletTextDataset(combined_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers = workers-1)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers = workers-1)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers = workers-1)

## Model creation

In [14]:
class TripletModelPL(pl.LightningModule):
    def __init__(self, bert_model, learning_rate=1e-4, wd = 0.01):
        super().__init__()
        self.bert_model = bert_model
        self.learning_rate = learning_rate
        self.weight_decay = wd

    def forward(self, A1_ids, A2_ids, B_ids, A1_mask, A2_mask, B_mask):
        A1_emb = self.bert_model(A1_ids, attention_mask=A1_mask).pooler_output
        A2_emb = self.bert_model(A2_ids, attention_mask=A2_mask).pooler_output
        B_emb = self.bert_model(B_ids, attention_mask=B_mask).pooler_output
        return A1_emb, A2_emb, B_emb

    def training_step(self, batch, batch_idx):
        pos_dist, neg_dist = self._get_triplet_distances(batch)
        loss = self.triplet_loss(pos_dist, neg_dist)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        pos_dist, neg_dist = self._get_triplet_distances(batch)
        loss = self.triplet_loss(pos_dist, neg_dist)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        pos_dist, neg_dist = self._get_triplet_distances(batch)
        loss = self.triplet_loss(pos_dist, neg_dist)
        accuracy = self.calculate_accuracy(pos_dist, neg_dist)
        self.log('test_loss', loss)
        self.log('test_accuracy', accuracy)
        return {'test_loss': loss, 'test_accuracy': accuracy}

    def calculate_accuracy(self, pos_dist, neg_dist):
        correct = (pos_dist < neg_dist).float()
        return correct.mean()

    def _get_triplet_distances(self, batch):
        A1_ids, A2_ids, B_ids = batch['anchor'], batch['positive'], batch['negative']
        A1_mask, A2_mask, B_mask = batch['anchor_mask'], batch['positive_mask'], batch['negative_mask']
        A1_emb, A2_emb, B_emb = self(A1_ids, A2_ids, B_ids, A1_mask, A2_mask, B_mask)
        pos_dist = F.pairwise_distance(A1_emb, A2_emb, 2)
        neg_dist = F.pairwise_distance(A1_emb, B_emb, 2)
        return pos_dist, neg_dist


    def triplet_loss(self, pos_dist, neg_dist, margin=0.5):
        loss = F.relu(pos_dist - neg_dist + margin)
        return loss.mean()

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=self.learning_rate,weight_decay = self.weight_decay)

In [15]:
# Create the model
Tripletmodel = TripletModelPL(bert_model, learning_rate = 0.00002)

In [16]:
logger = loggers.TensorBoardLogger("tb_logs", name="triplet_model")

# Checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath=str(os.path.join(os.getcwd(), 'checkpoints/')),   # Directory where models are saved
    filename='model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min'
)

# Early stopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=True,
    mode='min'
)

# Setup the trainer with added callbacks for checkpointing and early stopping
trainer = Trainer(
    max_epochs=10,
    callbacks=[checkpoint_callback, early_stopping_callback],
    logger=logger,
    accumulate_grad_batches=1,
    gradient_clip_val=100,
    gradient_clip_algorithm='value',
)

/home/cpsc452_sw2349/.conda/envs/cpsc552/lib/python3.12/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/cpsc452_sw2349/.conda/envs/cpsc552/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
# Train the model
trainer.fit(Tripletmodel, train_loader, val_loader)

/home/cpsc452_sw2349/.conda/envs/cpsc552/lib/python3.12/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/cpsc452_sw2349/.conda/envs/cpsc552/lib/python3 ...
You are using a CUDA device ('NVIDIA RTX A5000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/cpsc452_sw2349/.conda/envs/cpsc552/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /gpfs/gibbs/project/cpsc452/cpsc452_sw2349/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | T

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 36289.340


...

Metric val_loss improved by 1032.416 >= min_delta = 0.0. New best score: 49.712


Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved by 25.784 >= min_delta = 0.0. New best score: 23.929


Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved by 12.962 >= min_delta = 0.0. New best score: 10.967
`Trainer.fit` stopped: `max_epochs=10` reached.


In [18]:
# Test the model
trainer.test(Tripletmodel, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9639999866485596     │
│         test_loss         │    34.441890716552734     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 34.441890716552734, 'test_accuracy': 0.9639999866485596}]

In [19]:
def generate_embeddings(model, dataloader):
    model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['anchor']
            attention_mask = batch['anchor_mask']
            outputs = model(input_ids, input_ids, input_ids, attention_mask, attention_mask, attention_mask)
            embeddings.extend(outputs[0].cpu().numpy())  # Assuming outputs[0] are the embeddings
            labels.extend(batch['labels'].cpu().numpy())  # Now correctly accessing labels
    return np.array(embeddings), np.array(labels)


# Generate embeddings for the entire dataset
train_embeddings, train_labels = generate_embeddings(Tripletmodel, train_loader)
val_embeddings, val_labels = generate_embeddings(Tripletmodel, val_loader)
test_embeddings, test_labels = generate_embeddings(Tripletmodel, test_loader)

In [20]:
# Save embeddings and labels
np.save('train_embeddings.npy', train_embeddings)
np.save('train_labels.npy', train_labels)
np.save('val_embeddings.npy', val_embeddings)
np.save('val_labels.npy', val_labels)
np.save('test_embeddings.npy', test_embeddings)
np.save('test_labels.npy', test_labels)


In [21]:
# Load embeddings and labels
train_embeddings = np.load('train_embeddings.npy')
train_labels = np.load('train_labels.npy')
val_embeddings = np.load('val_embeddings.npy')
val_labels = np.load('val_labels.npy')
test_embeddings = np.load('test_embeddings.npy')
test_labels = np.load('test_labels.npy')

In [42]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out

# Define the input size, hidden layer size, and number of classes
input_size = train_embeddings.shape[1]
hidden_size = 100  # Example: 100 hidden units
num_classes = 3  # 3 classes

# Initialize the MLP
model = MLP(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)


In [43]:
# Convert numpy arrays to torch tensors
train_embeddings_tensor = torch.tensor(train_embeddings).float()
train_labels_tensor = torch.tensor(train_labels).long()
val_embeddings_tensor = torch.tensor(val_embeddings).float()
val_labels_tensor = torch.tensor(val_labels).long()
test_embeddings_tensor = torch.tensor(test_embeddings).float()
test_labels_tensor = torch.tensor(test_labels).long()

best_val_accuracy = 0
model_path = 'best_model.pth'

# Train the model
epochs = 300
for epoch in range(epochs):
    model.train()
    outputs = model(torch.tensor(train_embeddings).float())
    loss = criterion(outputs, torch.tensor(train_labels).long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(torch.tensor(val_embeddings).float())
        _, predicted = torch.max(val_outputs, 1)
        correct = (predicted == torch.tensor(val_labels).long()).sum().item()
        total = val_labels.shape[0]
        val_accuracy = 100 * correct / total

    # Save the model if it has the best validation accuracy so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), model_path)
        print(f"Saved new best model with Validation Accuracy: {val_accuracy}%")

    print(f'Epoch {epoch+1}: Train Loss: {loss.item()}, Validation Accuracy: {val_accuracy}%')

Saved new best model with Validation Accuracy: 33.333333333333336%
Epoch 1: Train Loss: 688.8220825195312, Validation Accuracy: 33.333333333333336%
Epoch 2: Train Loss: 1131.275146484375, Validation Accuracy: 33.333333333333336%
Epoch 3: Train Loss: 1781.82373046875, Validation Accuracy: 33.333333333333336%
Epoch 4: Train Loss: 1689.6619873046875, Validation Accuracy: 33.333333333333336%
Epoch 5: Train Loss: 1123.2041015625, Validation Accuracy: 33.333333333333336%
Epoch 6: Train Loss: 925.990478515625, Validation Accuracy: 33.333333333333336%
Epoch 7: Train Loss: 840.9747314453125, Validation Accuracy: 33.333333333333336%
Epoch 8: Train Loss: 493.4176940917969, Validation Accuracy: 33.333333333333336%
Epoch 9: Train Loss: 403.7736511230469, Validation Accuracy: 33.333333333333336%
Epoch 10: Train Loss: 546.0785522460938, Validation Accuracy: 33.333333333333336%
Epoch 11: Train Loss: 472.62811279296875, Validation Accuracy: 33.333333333333336%
Epoch 12: Train Loss: 241.84176635742188, 

In [44]:
# Load the best model
best_model = MLP(input_size=train_embeddings.shape[1], hidden_size=100, num_classes=3)
best_model.load_state_dict(torch.load(model_path))

# Use the best model for evaluation or further training
best_model.eval()
# Example: evaluate on test data
with torch.no_grad():
    test_outputs = best_model(torch.tensor(test_embeddings).float())
    _, predicted = torch.max(test_outputs, 1)
    correct = (predicted == torch.tensor(test_labels).long()).sum().item()
    total = test_labels.shape[0]
    test_accuracy = 100 * correct / total
    print(f'Test Accuracy: {test_accuracy}%')


Test Accuracy: 79.0%


## Method 2: KNN Classifier

In [60]:
knn = KNeighborsClassifier(n_neighbors=3)

# Train the classifier on the training data
knn.fit(train_embeddings, train_labels)

KNeighborsClassifier(n_neighbors=3)

In [61]:
# Predicting labels for validation set
val_predictions = knn.predict(val_embeddings)

print("Validation Accuracy: ", accuracy_score(val_labels, val_predictions))
print(classification_report(val_labels, val_predictions))

Validation Accuracy:  0.976
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       125
           1       0.96      0.97      0.96       125
           2       0.98      1.00      0.99       125

    accuracy                           0.98       375
   macro avg       0.98      0.98      0.98       375
weighted avg       0.98      0.98      0.98       375



In [62]:
# Predicting test set
test_predictions = knn.predict(test_embeddings)
print("Test Accuracy: ", accuracy_score(test_labels, test_predictions))
print(classification_report(test_labels, test_predictions))

Test Accuracy:  0.934
              precision    recall  f1-score   support

           0       0.85      0.98      0.91       500
           1       0.98      0.98      0.98       500
           2       0.99      0.84      0.91       500

    accuracy                           0.93      1500
   macro avg       0.94      0.93      0.93      1500
weighted avg       0.94      0.93      0.93      1500



## Method 3: SVM classifier

In [51]:
# Combine training and validation sets for training the final model
X_train = np.vstack((train_embeddings, val_embeddings))
y_train = np.hstack((train_labels, val_labels))

# Test set
X_test = test_embeddings
y_test = test_labels

In [52]:
#Feature Scaling
#SVMs are sensitive to unscaled features, so it is crucial to normalize or standardize your features:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [53]:
# Setting up the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': ['scale', 'auto'],  # Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
    'kernel': ['linear', 'rbf', 'poly']  # Type of the kernel
}

# Creating the classifier
svm_model = GridSearchCV(SVC(probability=True), param_grid, verbose=2, n_jobs=-1, cv=5)
svm_model.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly']},
             verbose=2)

In [55]:
print("Best parameters found:", svm_model.best_params_)
best_svm = svm_model.best_estimator_

Best parameters found: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}


In [71]:
y_pred = best_svm.predict(X_test_scaled)

# Evaluation metrics
print("Classification report:")
print(classification_report(y_test, y_pred, digits = 4))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Accuracy:", accuracy_score(y_test, y_pred))

Classification report:
              precision    recall  f1-score   support

           0     0.8619    0.9860    0.9198       500
           1     0.9920    0.9880    0.9900       500
           2     0.9907    0.8520    0.9161       500

    accuracy                         0.9420      1500
   macro avg     0.9482    0.9420    0.9420      1500
weighted avg     0.9482    0.9420    0.9420      1500

Confusion Matrix:
[[493   4   3]
 [  5 494   1]
 [ 74   0 426]]
Accuracy: 0.942


## Method 4: Desision Tree

In [63]:
# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Fit the model on the training data
dt_classifier.fit(train_embeddings, train_labels)

DecisionTreeClassifier(random_state=42)

In [64]:
# Predict on the validation set
val_predictions = dt_classifier.predict(val_embeddings)

# Generate classification report
print("Validation Classification Report:")
print(classification_report(val_labels, val_predictions))

# Confusion matrix
print("Validation Confusion Matrix:")
print(confusion_matrix(val_labels, val_predictions))

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       125
           1       0.97      0.97      0.97       125
           2       0.98      1.00      0.99       125

    accuracy                           0.98       375
   macro avg       0.98      0.98      0.98       375
weighted avg       0.98      0.98      0.98       375

Validation Confusion Matrix:
[[120   4   1]
 [  2 121   2]
 [  0   0 125]]


In [67]:
parameters = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), parameters, verbose=2, n_jobs=-1, cv=5)
grid_search.fit(train_embeddings, train_labels)

print("Best parameters found: ", grid_search.best_params_)
dt_best = grid_search.best_estimator_

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters found:  {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}


In [70]:
# Predict on the test set
test_predictions = dt_best.predict(test_embeddings)

# Generate classification report
print("Test Classification Report:")
print(classification_report(test_labels, test_predictions, digits = 4))

# Confusion matrix
print("Test Confusion Matrix:")
print(confusion_matrix(test_labels, test_predictions))

Test Classification Report:
              precision    recall  f1-score   support

           0     0.8899    0.9700    0.9282       500
           1     0.9723    0.9820    0.9771       500
           2     0.9800    0.8820    0.9284       500

    accuracy                         0.9447      1500
   macro avg     0.9474    0.9447    0.9446      1500
weighted avg     0.9474    0.9447    0.9446      1500

Test Confusion Matrix:
[[485   9   6]
 [  6 491   3]
 [ 54   5 441]]


## Method 5: Logistics Regression

In [104]:
scaler = StandardScaler()
train_embeddings_scaled = scaler.fit_transform(train_embeddings)
val_embeddings_scaled = scaler.transform(val_embeddings)
test_embeddings_scaled = scaler.transform(test_embeddings)

In [74]:
# Define the model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(train_embeddings_scaled, train_labels)

LogisticRegression(max_iter=1000)

In [76]:
# Predictions
val_predictions = model.predict(val_embeddings_scaled)
test_predictions = model.predict(test_embeddings_scaled)

# Evaluation metrics
print("Validation Accuracy:", accuracy_score(val_labels, val_predictions))
print("Test Accuracy:", accuracy_score(test_labels, test_predictions))
print("\nValidation Classification Report:\n", classification_report(val_labels, val_predictions, digits = 4))
print("\nTest Classification Report:\n", classification_report(test_labels, test_predictions, digits = 4))

Validation Accuracy: 0.9813333333333333
Test Accuracy: 0.9453333333333334

Validation Classification Report:
               precision    recall  f1-score   support

           0     0.9918    0.9680    0.9798       125
           1     0.9683    0.9760    0.9721       125
           2     0.9843    1.0000    0.9921       125

    accuracy                         0.9813       375
   macro avg     0.9814    0.9813    0.9813       375
weighted avg     0.9814    0.9813    0.9813       375


Test Classification Report:
               precision    recall  f1-score   support

           0     0.8766    0.9800    0.9254       500
           1     0.9859    0.9820    0.9840       500
           2     0.9865    0.8740    0.9268       500

    accuracy                         0.9453      1500
   macro avg     0.9497    0.9453    0.9454      1500
weighted avg     0.9497    0.9453    0.9454      1500



## Baseline Mobile Bert

In [78]:
model_name = 'google/mobilebert-uncased'
model = MobileBertForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = MobileBertTokenizer.from_pretrained(model_name)

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [83]:
def encode_texts(tokenizer, texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=512, return_tensors="pt")

# Assuming combined_test is your DataFrame and Text is the column with texts
encoded_test = encode_texts(tokenizer, combined_test['Text'].tolist())

In [84]:
with torch.no_grad():  # This will disable gradient calculations
    outputs = model(**encoded_test)
    predictions = torch.argmax(outputs.logits, dim=-1)

In [86]:
combined_test

,Text,Label
0,I have two questions for you. One: whose shoul...,Mixed
1,"On October 4, 1957, the world watched in awe a...",Mixed
2,Failure to Failure Equals Success: The Importa...,PureAI
3,"Education is a life-long process, and I strong...",PureAI
4,I strongly believe that arts education should ...,PureAI
...,...,...
1495,"Hello. This is my first trip, first time in li...",Mixed
1496,"We require a weather service for water, but it...",Mixed
1497,I think the state senator should change the el...,PureHuman
1498,"OMG, so u wanna know how long it'll take to dr...",PureHuman


In [92]:
label_mapping = {'PureAI': 0, 'PureHuman': 1, 'Mixed': 2}
gt_labels = combined_test['Label'].map(label_mapping)

print(classification_report(gt_labels, predictions.numpy(), target_names=['PureAi', 'PureHuman', 'Mixed'], digits = 4))

              precision    recall  f1-score   support

      PureAi     0.3333    1.0000    0.5000       500
   PureHuman     0.0000    0.0000    0.0000       500
       Mixed     0.0000    0.0000    0.0000       500

    accuracy                         0.3333      1500
   macro avg     0.1111    0.3333    0.1667      1500
weighted avg     0.1111    0.3333    0.1667      1500



/home/cpsc452_sw2349/.conda/envs/cpsc552/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/cpsc452_sw2349/.conda/envs/cpsc552/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/cpsc452_sw2349/.conda/envs/cpsc552/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f